In [7]:
import pandas as pd
from IPython.display import display
import mysql.connector
import os
from dotenv import load_dotenv

# Extract Data

In [2]:
df_admin = pd.read_csv('Data_Source(dummy)/admin.csv')
df_applications = pd.read_csv('Data_Source(dummy)/applications.csv')
df_article = pd.read_csv('Data_Source(dummy)/article.csv')
df_comment = pd.read_csv('Data_Source(dummy)/comment.csv')
df_customer_bookmark_article = pd.read_csv('Data_Source(dummy)/customer_bookmark_article.csv')
df_customer_bookmark_fundraising = pd.read_csv('Data_Source(dummy)/customer_bookmark_fundraising.csv')
df_customer_bookmark_volunteer = pd.read_csv('Data_Source(dummy)/customer_bookmark_volunteer.csv')
df_customers = pd.read_csv('Data_Source(dummy)/customers.csv')
df_donations = pd.read_csv('Data_Source(dummy)/donations.csv')
df_fundraising_categories = pd.read_csv('Data_Source(dummy)/fundraising_categories.csv')
df_fundraisings = pd.read_csv('Data_Source(dummy)/fundraisings.csv')
df_likes_comment = pd.read_csv('Data_Source(dummy)/likes_comment.csv')
df_likes_fundraising = pd.read_csv('Data_Source(dummy)/likes_fundraising.csv')
df_organization = pd.read_csv('Data_Source(dummy)/organization.csv')
df_testimoni_volunteer = pd.read_csv('Data_Source(dummy)/testimoni_volunteer.csv')
df_volunteer_vacancies = pd.read_csv('Data_Source(dummy)/volunteer_vacancies.csv')
df_wish_fundraising = pd.read_csv('Data_Source(dummy)/wish_fundraising.csv')

In [9]:
def ingestion(table):
    load_dotenv()
    connection = mysql.connector.connect(
    host=os.getenv('host'),
    user='root',
    password=os.getenv('password'),
    database='capstone5'
    )
    query = f"SELECT * FROM {table}"  
    df = pd.read_sql(query, connection)
    connection.close()
    return df

In [15]:
df_applications = ingestion('applications')
df_articles = ingestion('articles')
df_comments = ingestion('comments')
df_donations = ingestion('donations')
df_donation_comments = ingestion('donation_comments')
df_fundraising_categories = ingestion('fundraising_categories')
df_fundraisings = ingestion('fundraisings')

df_customers = ingestion('users')
df_likes_comments = ingestion('likes_comments')
df_testimoni_volunteers = ingestion('testimoni_volunteers')
df_volunteer_vacancies = ingestion('volunteers')

C:\Users\SINTA\AppData\Local\Temp\ipykernel_8848\3238963636.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
C:\Users\SINTA\AppData\Local\Temp\ipykernel_8848\3238963636.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
C:\Users\SINTA\AppData\Local\Temp\ipykernel_8848\3238963636.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
C:\Users\SINTA\AppData\Local\Temp\ipykernel_8848\3238963636.py:10: UserWarning: pa

In [16]:
display(df_applications)
display(df_article)
display(df_comments)
display(df_donations)
display(df_donation_comments)
display(df_fundraising_categories)
display(df_fundraisings)
display(df_customers)
display(df_likes_comments)
display(df_testimoni_volunteers)
display(df_volunteer_vacancies)

,id,created_at,updated_at,deleted_at,ig_image_url,yt_image_url,customer_id,vacancy_id,reason,age


,id,admin_id,title,content,image_url,created_at,updated_at,deleted_at
0,1,3,Another sort measure kind assume believe.,Pattern yourself raise chance. Discussion rich...,https://dummyimage.com/588x338,2023-07-12 12:19:40,2023-10-25 12:19:40,2024-02-19 12:19:40
1,2,3,Field want car debate position.,Nearly back since apply. Executive Congress fi...,https://www.lorempixel.com/397/931,2023-07-31 12:19:40,2024-03-31 12:19:40,2023-10-09 12:19:40
2,3,2,State break sometimes strong author bar notice.,Some high through. Short concern politics scen...,https://dummyimage.com/970x357,2024-01-16 12:19:40,2024-05-28 12:19:40,2024-01-23 12:19:40
3,4,2,Art administration politics responsibility sound.,Usually blue director look similar Mrs. Their ...,https://dummyimage.com/922x118,2023-08-26 12:19:40,2024-01-26 12:19:40,2023-12-22 12:19:40
4,5,2,Mother building mission television soon article.,Another those cup season. Until each note stud...,https://dummyimage.com/490x921,2024-04-15 12:19:40,2024-05-07 12:19:40,2023-11-26 12:19:40
5,6,3,Continue term city hold choice wall.,Hotel develop spring free. Choose save green c...,https://www.lorempixel.com/724/774,2023-07-02 12:19:40,2024-03-05 12:19:40,NaN
6,7,3,History race another thought.,Mind answer arm. Build inside listen green bor...,https://www.lorempixel.com/951/65,2023-07-27 12:19:40,2024-01-30 12:19:40,NaN
7,8,3,Simple draw none power.,Possible degree stock herself soldier. City sa...,https://www.lorempixel.com/444/103,2023-08-23 12:19:40,2023-11-10 12:19:40,2024-04-14 12:19:40
8,9,2,Yes suddenly dark.,Look view side. Sure safe number term. These f...,https://dummyimage.com/116x256,2023-12-02 12:19:40,2024-03-11 12:19:40,NaN
9,10,3,Middle wrong dark economy score risk board.,Position per evidence happy remain appear alon...,https://www.lorempixel.com/615/921,2023-09-23 12:19:40,2024-02-29 12:19:40,NaN


,id,created_at,updated_at,deleted_at,customer_id,article_id,comment


,id,created_at,updated_at,deleted_at,user_id,amount,status,code,fundraising_id,payment_url,payment_method


,id,created_at,updated_at,deleted_at,donation_id,comment,total_likes


...

,id,created_at,updated_at,deleted_at,fundraising_category_id,organization_id,title,image_url,description,status,goal_amount,current_progress,start_date,end_date


,id,created_at,updated_at,deleted_at,fullname,username,email,password,avatar,reset_token,reset_token_expire
0,1,2024-06-03 10:45:11.896,2024-06-03 10:45:11.896,None,John Doe,johndoe,newaccount@test.com,123123,https://res.cloudinary.com/dvrhf8d9t/image/upl...,,None
1,2,2024-06-03 19:14:59.320,2024-06-03 19:14:59.320,None,Kris,Kris9,testaja@test.com,123123,https://res.cloudinary.com/dvrhf8d9t/image/upl...,,None
2,3,2024-06-03 12:29:27.804,2024-06-03 12:29:27.804,None,Kris2,Kris92,testaja2@test.com,123123,https://res.cloudinary.com/dvrhf8d9t/image/upl...,,None


,id,created_at,updated_at,deleted_at,customer_id,comment_id


,id,created_at,updated_at,deleted_at,customer_id,volunteer_id,testimoni,rating,date


,id,created_at,updated_at,deleted_at,organization_id,title,content_activity,location,date,target_volunteer,registered_volunteer,registration_deadline,image_url


# Transform

## Tabel Fakta

### 1. FactDonationTransaction

In [ ]:
# Buat Struktur kolom df_fact_donation_transaction
columns = ['id', 'donation_id', 'fundraising_id', 'customer_id', 'nominal', 'goal_amount', 'fundraising_categories_id', 'organization_id', 'donation_created_at']
df_fact_donation_transaction = pd.DataFrame(columns=columns)

In [ ]:
# mengambil data yang sukses di df_donation 
df_donation_success = df_donations.loc[df_donations['status'] == 'success']
df_donation_success = df_donation_success.reset_index(drop=True)

In [ ]:
# mengisi data pada dari kolom df_donation
df_fact_donation_transaction['id'] = range(1, len(df_donation_success) + 1)
df_fact_donation_transaction['donation_id'] = df_donation_success['id']
df_fact_donation_transaction['fundraising_id'] = df_donation_success['fundraisings_id']
df_fact_donation_transaction['customer_id'] = df_donation_success['customer_id']
df_fact_donation_transaction['nominal'] = df_donation_success['nominal']
df_fact_donation_transaction['donation_created_at'] = df_donation_success['created_at']

In [ ]:
# merge df_fundraising
df_merge_fact_fundraising = pd.merge(df_fact_donation_transaction, df_fundraisings, left_on='fundraising_id', right_on='id', how='left')

# inisialisasi data di df_fact_donation_transaction
df_fact_donation_transaction['goal_amount'] = df_merge_fact_fundraising['goal_amount_y']
df_fact_donation_transaction['fundraising_categories_id'] = df_merge_fact_fundraising['fundraising_categories_id_y']
df_fact_donation_transaction['organization_id'] = df_merge_fact_fundraising['organization_id_y']
df_fact_donation_transaction

### 2. Fact Volunteer Applications

In [ ]:
# Buat Struktur kolom df_fact_volunteer_applications
columns = ['id', 'application_id', 'volunteer_vacancies_id', 'customer_id', 'organization_id', 'application_created_at']
df_fact_volunteer_applications = pd.DataFrame(columns=columns)

In [ ]:
# mengisi data pada dari kolom df_application
df_fact_volunteer_applications['id'] = range(1, len(df_applications) + 1)
df_fact_volunteer_applications['application_id'] = df_applications['id']
df_fact_volunteer_applications['volunteer_vacancies_id'] = df_applications['vacancy_id']
df_fact_volunteer_applications['customer_id'] = df_applications['customer_id']
df_fact_volunteer_applications['application_created_at'] = df_applications['create_at']
df_fact_volunteer_applications

In [ ]:
# merge df_fundraising
df_merge_fact_volunteer = pd.merge(df_fact_volunteer_applications, df_volunteer_vacancies, left_on='volunteer_vacancies_id', right_on='id', how='left')

# inisialisasi data di df_fact_volunteer_applications
df_fact_volunteer_applications['organization_id'] = df_merge_fact_volunteer['organization_id_y']
df_fact_volunteer_applications

### 3. Fact Volunteer Testimoni

In [ ]:
# Buat Struktur kolom df_fact_volunteer_testimoni
columns = ['id', 'customer_id', 'volunteer_vacancies_id', 'testimoni_volunteer_id', 'rating']
df_fact_volunteer_testimoni = pd.DataFrame(columns=columns)

In [ ]:
# mengisi data pada dari kolom df_application
df_fact_volunteer_testimoni['id'] = range(1, len(df_applications) + 1)
df_fact_volunteer_testimoni['customer_id'] = df_testimoni_volunteer['customer_id']
df_fact_volunteer_testimoni['volunteer_vacancies_id'] = df_testimoni_volunteer['volunteer_id']
df_fact_volunteer_testimoni['testimoni_volunteer_id'] = df_testimoni_volunteer['id']
df_fact_volunteer_testimoni['rating'] = df_testimoni_volunteer['rating']
df_fact_volunteer_applications

### 4. Fact Article Popular

In [ ]:
# Buat Struktur kolom df_fact_articel_popular
columns = ['id', 'article_id', 'comment_id', 'customer_id', 'total_like', 'comment_created_at']
df_fact_article_popular = pd.DataFrame(columns=columns)

In [ ]:
# mengisi data pada dari kolom df_comment
df_fact_article_popular['id'] = range(1, len(df_comment) + 1)
df_fact_article_popular['article_id'] = df_comment['article_id']
df_fact_article_popular['comment_id'] = df_comment['id']
df_fact_article_popular['customer_id'] = df_comment['customer_id']
df_fact_article_popular['total_like'] = df_comment['total_like']
df_fact_article_popular['comment_created_at'] = df_comment['created_at']
df_fact_article_popular

### 5. Fact Bookmark Fundraising

In [ ]:
df_fact_bookmark_fundraising = df_customer_bookmark_fundraising.drop(['deleted_at', 'updated_at'], axis=1)
df_fact_bookmark_fundraising = df_fact_bookmark_fundraising.rename(columns={'id':'bookmark_id'})
df_fact_bookmark_fundraising['id'] = range(1, len(df_fact_bookmark_fundraising) + 1)
df_fact_bookmark_fundraising.insert(0, 'id', df_fact_bookmark_fundraising.pop('id'))
df_fact_bookmark_fundraising

### 6. Fact Bookmark Volunteer Vacancies

In [ ]:
df_fact_bookmark_volunteer_vacancies = df_customer_bookmark_volunteer.drop(['deleted_at', 'updated_at'], axis=1)
df_fact_bookmark_volunteer_vacancies = df_fact_bookmark_volunteer_vacancies.rename(columns={'id':'bookmark_id'})
df_fact_bookmark_volunteer_vacancies['id'] = range(1, len(df_fact_bookmark_volunteer_vacancies) + 1)
df_fact_bookmark_volunteer_vacancies.insert(0, 'id', df_fact_bookmark_volunteer_vacancies.pop('id'))
df_fact_bookmark_volunteer_vacancies

## Tabel Dimensi

In [40]:
dim_fundraisings = df_fundraisings.drop(['fundraising_categories_id','organization_id','updated_at'], axis=1)
dim_fundraising_categories = df_fundraising_categories[['id','category_name','update_at']]
dim_donation = df_donations.drop(['fundraisings_id', 'customer_id','updated_at'], axis=1)
dim_organization = df_organization
dim_customer = df_customers
dim_volunteer_applictaion = df_applications.drop(['customer_id','vacancy_id','update_at'], axis=1)
dim_volunteer_vacancies = df_volunteer_vacancies.drop(['organization_id','updated_at'], axis=1)
dim_testimoni_volunteer = df_testimoni_volunteer.drop(['customer_id','volunteer_id','updated_at'], axis=1)
dim_article = df_article
dim_comment = df_comment.drop(['customer_id','article_id','updated_at'], axis=1)
dim_bookmark_fundraising = df_customer_bookmark_fundraising.drop(['fundraising_id','customer_id','updated_at'], axis=1)
dim_bookmark_volunter_vacancies = df_customer_bookmark_volunteer.drop(['volunteer_vacancies_id','customer_id','updated_at'], axis=1)


# Load

In [ ]:
df_fact_donation_transaction.to_csv("Data_Loaded/Fact/fact_donation_transaction.csv", index=False)
df_fact_volunteer_applications.to_csv("Data_Loaded/Fact/fact_volunteer_applications.csv", index=False)
df_fact_volunteer_testimoni.to_csv("Data_Loaded/Fact/fact_volunteer_testimoni.csv", index=False)
df_fact_article_popular.to_csv("Data_Loaded/Fact/fact_article_popular.csv", index=False)


